# MMDA Prohibited roads for e-bike

The MMDA Regulation No. 24-022 prohibited roads for e-bike are as follows:

1. **C1:** Recto Avenue
2. **C2:** Pres. Quirino Avenue
3. **C3:** Araneta Avenue
4. **C4:** EDSA
5. **C5:** Katipunan/CP Garcia
6. **C6:** Southeast Metro Manila Expressway
7. **R1:** Roxas Boulevard
8. **R2:** Taft Avenue
9. **R3:** Osmeña Highway
10. **R4:** Shaw Boulevard
11. **R5:** Ortigas Avenue
12. **R6:** Magsaysay Blvd./Aurora Blvd.
13. **R7:** Quezon Ave./Commonwealth Ave.
14. **R8:** A. Bonifacio Ave.
15. **R9:** Rizal Ave.
16. **R10:** Del Pan/Marcos Highway/McArthur Highway
17. Elliptical Road
18. Mindanao Avenue
19. Marcos Highway
20. Boni Avenue (Mandaluyong)
21. España Blvd.

We'll have to map out these roads/edges into a set of edges `prohibited_roads` so that we can remove them from our main graph, and also embed the `prohibited_roads` edges in our map (to show that our Dijkstra routes avoid them)

## Issues

The roads/edges in OpenStreetmaps are cut-off into multiple parts. A stretch of road (say Boni Avenue) corresponds to multiple edges in our graph which can each have a different name (or have a list of names instead of a single one, or worse have no name at all)

- For each road in the MMDA prohibited roads, inspect the main graph edges using the `.explore` function to check what names were used in the OSM graph for that road (e.g. C4: EDSA have different names in our graph: )



In [18]:
import osmnx as ox
import networkx as nx
import pandas as pd
weight = "length"

In [19]:
# Load Metro Manila graph
G = ox.io.load_graphml(filepath="data/metro_drive.graphml")
nodes, edges = ox.graph_to_gdfs(G)

In [ ]:
# This maps out all the edges in our graph. Useful for manual insepction of roads but is very laggy
#edges.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 3})

In [ ]:
#  Highway is an attribute of OSM roads/edges, find all which are either motorway, trunk, or primary since they are the most important highway tags/categories
cf = '["highway"~"motorway|trunk|primary"]' # https://wiki.openstreetmap.org/wiki/Key:highway
primary_G = ox.graph_from_place("Metro Manila, Philippines", network_type="drive", custom_filter=cf)
primary_nodes, primary_edges = ox.graph_to_gdfs(G)

## Filter for highway types: motorway, trunk, primary is a shortcut to quickly get most of the prohibited roads

Since the MMDA prohibited roads for e-bikes are national, circumferential, and radial roads (roads which could be deemed as the most *important* roads) we can easily get most of them by filtering the OpenStreetMap graph edges for those that have highway type motorway, trunk, or primary (which are the most *important* highway tags in OpenStreetMaps)

Hence, we can start creating our `prohibited_roads` by filling it with all the roads/edge with highway type motorway/trunk/primary.

However, there would still be some roads/edges we could have missed: either their highway tags did not match motorway/trunk/primary or had no highway tags altogether. We have to add them to our `prohibited_roads`

Also, some roads has the highway tag motorway/trunk/primary but are not part of the MMDA prohibited roads. We have to remove them from our `prohibited_roads`

In [ ]:
#primary_edges.explore(tiles="cartodbdarkmatter", style_kwds={"weight": 5, "opacity": 0.5})

Most of the MMDA prohibited roads are already captured by filtering for highway type motorway/trunk/primary. The first one I noticed not part of this set of motorway/trunk/primary road edges is Boni Avenue. For the rest of the MMDA prohibited roads, inspect if it is in this set of primary edges (search online for some reference on where the stretch of the road is)

Already checked the following are part of the `primary_edges`:

- Elliptical Road
- Marcos Highway
- Espana Boulevard
- Mindanao Avenue

For the rest of the Circumferential (C1-C6) and Radial (R1-R10) roads, refer to this map to get a rouch sketch https://www.google.com/maps/d/viewer?mid=1XjNUjlg6KEnrl9OkZ_0PDKj5ay4&hl=en_US

Note that some Radial and Circumferential roads go beyond the land boundary of Metro Manila. We can just disregard them

In [ ]:
# Initialize our prohibited roads with this set of primary edges
prohibited_roads = primary_edges.copy(deep=True)

### Some roads/edges have null names

Just let them be for now

In [ ]:
null_edges = edges[pd.isnull(edges.name)]
#null_edges.explore(tiles="cartodbdarkmatter")

## Example road inspections

### Map out Boni Avenue

Boni Avenue is part of the MMDA prohibited roads but was not part of our initial filtering with highway types of motorway/trunk/primary (since from inspection it has the `secondary` type) so we have to manually identify the roads/edges forming Boni Avenue

In [ ]:
boni_edges = edges[edges.name == "Boni Avenue"]
#boni_edges.explore(tiles="cartodbdarkmatter")

### Quezon City: East Avenue, North Avenue, West Avenue and Timog Avenue

Our initial filtering with highway types of motorway/trunk/primary in `primary_edges` contains these 4 Quezon City avenues, but they are not part of the prohibited roads. We have to remove them from our set of `prohibited_roads`

In [ ]:
# Filtering using isin
qc_edges = edges[edges['name'].isin(["East Avenue", "Timog Avenue", "North Avenue", "West Avenue"])]
#qc_edges.explore( tiles="cartodbdarkmatter")

## Stitching it all together

Hence from our `prohibited_roads` which currently holds all the `primary_edges` which just contains all roads with highway type motorway/trunk/primary, we have to:

- Add `boni_edges`
- Remove `qc_edges`
- For the rest of the C1-C6 roads and R1-R10 roads, check if it was successfully capture
    - **TODO**: Add all missing edges (if there are missing edges)
    - **TODO**: Remove all edges that are not actually prohibited (if there are edges wrongfully added from `primary_edges`)

Not sure what the pandas functions are to add/remove edges to `prohibited_roads`

In [ ]:
# Add boni edges, remove qc edges, and all the other things to prohibited roads
# prohibited_roads = pd.concat... some filtering to join boni_edges
# prohibited_roads = prohibited_roads[some filtering to remove qc_edges]

## Create new Metro Manila edges for the prohibited roads, and the Metro Manila edges without the prohibited roads

In [ ]:
# Metro Manila edges for the prohibited roads
metro_edges_prohibited_roads = edges[~edges['name'].isin(prohibited_roads['name'])]

# Metro Manila edges without the prohibited roads 
metro_edges_without_prohibited = edges[~edges['name'].isin(prohibited_roads['name'])]

## Output with OSMnx

Now that we have all the edges of Metro Manila without the prohibited roads, we need to create an new graph using the original `nodes` and the new edges `metro_edges_without_prohibited`. Then save it to disk


In [ ]:
# Export graph of Metro Manila without the prohibited roads
# metro_graph_without_prohibited = ox.graph_from_gdfs(nodes, metro_edges_without_prohibited)

# Export graph of primary nodes and primary edges for the prohibited roads (since we're going to map the prohibited roads)
# metro_graph_prohibited = ox.graph_from_gdfs(primary_nodes, metro_edges_prohibited_roads)

In [ ]:
# Save the graph to disk
# ox.io.save_graphml(metro_graph_without_prohibited, filepath="data/metro_graph_without_prohibited.graphml")
# ox.io.save_graphml(metro_graph_prohibited, filepath="data/metro_graph_prohibited.graphml")